In [1]:
import numpy as np
import sys
import chainer

In [2]:
sys.path.append("../")

In [3]:
import ritc

In [4]:
x = np.random.normal(0, 1, (5, 4)).astype(np.float32)
t = np.array([0, 1, 2, 3, 2], dtype=np.int32)

In [5]:
import ritc.functions as F
import ritc.networks as N

In [6]:
train, test = chainer.datasets.get_mnist()
x_train, t_train = train._datasets
x_test, t_test = test._datasets

In [7]:
class Model(ritc.Chain):
    def __init__(self):
        super(Model, self).__init__()
        
        self.linear1 = N.Linear(784, 256)
        self.linear2 = N.Linear(256, 128)
        self.linear3 = N.Linear(128, 10)
        self.add_net(self.linear1)
        self.add_net(self.linear2)
        self.add_net(self.linear3)
    
    def __call__(self, x, t):
        y = self.forward(x)
        self.y = y
        loss = F.softmax_cross_entropy(y, t)
        return loss
        
    def forward(self, x):
        h = self.linear1(x)
        h = F.relu(h)
        h = self.linear2(h)
        h = F.relu(h)
        y = self.linear3(h)
        return y

In [8]:
model = Model()
optimizer = ritc.base.Optimizer(1e-3)
optimizer.setup(model)

In [9]:
batch_size = 32
epoch_num = 100

In [10]:
for epoch in range(epoch_num):
    for i in range(len(train) // batch_size):
        x_batch = x_train[i*batch_size:(i+1)*batch_size]
        t_batch = t_train[i*batch_size:(i+1)*batch_size]
        loss = model(x_batch, t_batch)
        if i % 100 == 0:
            print(epoch, "\t", i, "\t", loss.data)
        model.cleargrads()
        loss.backward()
        optimizer.optimize()
        
    accuracy = 0
    for i in range(len(test) // batch_size + 1):
        x_batch = x_train[i*batch_size:min((i+1)*batch_size, len(test))]
        t_batch = t_train[i*batch_size:min((i+1)*batch_size, len(test))]
        predict = model.forward(x_batch).data
        predict = np.argmax(predict, axis=1)
        acc_batch = sum([1 if int(p)==int(t) else 0 for p, t in zip(predict, t_batch)])
        accuracy += acc_batch
    print("accuracy: ", accuracy/len(test))

0 	 0 	 5.888277
0 	 100 	 5.7724237
0 	 200 	 5.63898
0 	 300 	 5.6628046
0 	 400 	 5.6341486
0 	 500 	 5.620238
0 	 600 	 5.516922
0 	 700 	 5.4799557
0 	 800 	 5.464382
0 	 900 	 5.4634137
0 	 1000 	 5.4479656
0 	 1100 	 5.513556
0 	 1200 	 5.441391
0 	 1300 	 5.517556
0 	 1400 	 5.559373
0 	 1500 	 5.421943
0 	 1600 	 5.4028
0 	 1700 	 5.4247837
0 	 1800 	 5.413541
accuracy:  0.7121
1 	 0 	 5.4590807
1 	 100 	 5.4412327
1 	 200 	 5.355001
1 	 300 	 5.4130425
1 	 400 	 5.4391756
1 	 500 	 5.474531
1 	 600 	 5.3335094
1 	 700 	 5.3340926
1 	 800 	 5.312192
1 	 900 	 5.310338
1 	 1000 	 5.3446693
1 	 1100 	 5.406987
1 	 1200 	 5.3440886
1 	 1300 	 5.4306107
1 	 1400 	 5.45319
1 	 1500 	 5.3172064
1 	 1600 	 5.305264
1 	 1700 	 5.347966
1 	 1800 	 5.3335686
accuracy:  0.7959
2 	 0 	 5.372491
2 	 100 	 5.3660607
2 	 200 	 5.2895813
2 	 300 	 5.3382444
2 	 400 	 5.3746767
2 	 500 	 5.419143
2 	 600 	 5.274485
2 	 700 	 5.277854
2 	 800 	 5.254597
2 	 900 	 5.2613287
2 	 1000 	 5.302761
2

20 	 600 	 5.0836563
20 	 700 	 5.0784984
20 	 800 	 5.077696
20 	 900 	 5.101363
20 	 1000 	 5.1247168
20 	 1100 	 5.1521435
20 	 1200 	 5.149386
20 	 1300 	 5.2021837
20 	 1400 	 5.1771946
20 	 1500 	 5.094651
20 	 1600 	 5.063889
20 	 1700 	 5.1078906
20 	 1800 	 5.124996
accuracy:  0.9283
21 	 0 	 5.1330748
21 	 100 	 5.169709
21 	 200 	 5.0961514
21 	 300 	 5.0831976
21 	 400 	 5.160054
21 	 500 	 5.2002544
21 	 600 	 5.080508
21 	 700 	 5.074833
21 	 800 	 5.0755863
21 	 900 	 5.09885
21 	 1000 	 5.1221094
21 	 1100 	 5.1481833
21 	 1200 	 5.1466417


KeyboardInterrupt: 

In [23]:
x, t = test[6]

In [24]:
print(np.argmax(model.forward(x).data), t)

4 4


In [6]:
l1 = N.Linear(4, 6)
l2 = N.Linear(6, 4)

In [7]:
h = l1(x)
h = F.relu(h)
y = l2(h)
loss = F.softmax_cross_entropy(y, np.array([0, 1, 2, 3, 2], dtype=np.int32))

In [8]:
loss

Variable(28.993988)

In [ ]:
loss.backward()

In [11]:
id(l1)

47753470360